### Stock Market Trading with reinforcement Learning:-

++ Neural Netwroks, Deep-Q-Learning, Tensorflow 2.0 ,Google collab 

### Dependencies and environment setup:-

In [3]:
!pip install tensorflow-gpu==2.0.0.alpha0
!pip install pandas-datareader

In [0]:
# Importing project dependencies.

import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [5]:
tf.__version__

'2.0.0-alpha0'

### Building the RL trade network


In [0]:
class TraderNet():
  def __init__(self,state_size, action_space=3, model_name="TraderNet"):
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name

    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995

    self.model = self.model_builder()
  
  def model_builder(self):

    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))

    model.add(tf.keras.layers.Dense(units=64, activation='relu'))

    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))

    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001)) # mse, as regression model.
    
    return model

  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
  
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  def batch_train(self, batch_size):
    batch = []
    
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
      
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay


### Model Preprocessing

In [0]:
#Helper functions
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [0]:
#Price format function

def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

In [0]:
#Dataset loader

def dataset_loader(stock_name):
  
  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']

  return close

In [0]:
#State creator

def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

In [0]:
#Loading Dataset
stock_name = "AAPL"
data = dataset_loader(stock_name)

### Training TraderNet

In [0]:
#Setting hyper parameters

window_size = 10
episodes = 1000

batch_size = 32
data_samples = len(data) - 1

In [15]:
#Defining the Trader model

trader = TraderNet(window_size)
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                352       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,171
Trainable params: 11,171
Non-trainable params: 0
_________________________________________________________________


### Training TraderNet

In [0]:
for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("TraderNet bought: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("TraderNet sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))

  0%|          | 0/2461 [00:00<?, ?it/s]

Episode: 1/1000
TraderNet bought:  $ 30.572857
TraderNet sold:  $ 30.625713  Profit: $ 0.052856
TraderNet bought:  $ 30.282858
TraderNet sold:  $ 30.719999  Profit: $ 0.437141
TraderNet bought:  $ 29.420000
TraderNet bought:  $ 29.697144
TraderNet bought:  $ 27.818571
TraderNet bought:  $ 28.461428
TraderNet sold:  $ 27.435715  Profit: - $ 1.984285
TraderNet sold:  $ 27.922857  Profit: - $ 1.774286
TraderNet sold:  $ 28.625713  Profit: $ 0.807142
TraderNet bought:  $ 29.057142
TraderNet sold:  $ 28.935715  Profit: $ 0.474287
TraderNet bought:  $ 28.990000


  1%|▏         | 33/2461 [00:00<01:03, 38.29it/s]

TraderNet sold:  $ 28.631428  Profit: - $ 0.425714
TraderNet bought:  $ 28.151428


  1%|▏         | 35/2461 [00:01<02:21, 17.09it/s]

...

 22%|██▏       | 540/2461 [01:11<03:59,  8.00it/s]

TraderNet bought:  $ 73.769997
TraderNet bought:  $ 74.629997


 22%|██▏       | 542/2461 [01:11<04:02,  7.90it/s]

TraderNet sold:  $ 75.108574  Profit: $ 1.338577
TraderNet sold:  $ 76.487144  Profit: $ 1.857147


 23%|██▎       | 556/2461 [01:13<03:48,  8.34it/s]